# Boxplots

## Script setup

In [35]:
# Third-party and standard modules
from typing import Optional
import numpy as np
import pandas as pd
import seaborn
import matplotlib.pyplot as plt
import pytz
from itertools import repeat
from importlib import reload

# User-defined modules.
# The reload() function is needed to update modules after
# changes are made to their files.
from modules import dataframe_manip as dfm
from modules import misc, schemas, College1AcPtTimeSeries
# The reload() function is needed to update modules after
# changes are made to their files.
dfm = reload(dfm)
misc = reload(misc)
schemas = reload(schemas)
College1AcPtTimeSeries = reload(College1AcPtTimeSeries)

## Script parameters

The following values are this script's parameters.

In [36]:
filepath = './cisco_wifi_data_until_20190513.csv'
timezone = pytz.timezone('US/Pacific')
schema: schemas.AcPtTimeSeries =\
    College1AcPtTimeSeries.College1AcPtTimeSeries
buildings = [
    'SCC', 'HAHN', 'CROOKSHANK', 'THATHCHER', 'CLARK',
    'CARNEGIE', 'MASON', 'PEARSON', 'ALEXANDER', 'ITB'
]

## Data Processing

In [37]:
# A time-series dataframe holding occupancy counts
# for many access points (columns).
occ: pd.DataFrame = dfm.csv_to_timeseries_df(
    filepath=filepath, timezone=timezone
)

In [38]:
acpts = occ.columns.to_list()
acpts.sort()

buildings_in_cols = [False] * len(buildings)
for bi in range(0, len(buildings)):
    for a in acpts:
        if buildings[bi] in a:
            buildings_in_cols[bi] = True
            break

In [42]:
print(buildings)
print(buildings_in_cols)
print(list(zip(buildings, buildings_in_cols)))
print(acpts)

['SCC', 'HAHN', 'CROOKSHANK', 'THATHCHER', 'CLARK', 'CARNEGIE', 'MASON', 'PEARSON', 'ALEXANDER', 'ITB']
[True, True, False, False, True, True, True, False, True, True]
[('SCC', True), ('HAHN', True), ('CROOKSHANK', False), ('THATHCHER', False), ('CLARK', True), ('CARNEGIE', True), ('MASON', True), ('PEARSON', False), ('ALEXANDER', True), ('ITB', True)]
['188-7TH-1', '188-7TH-2', '188-7TH-GARAGE', 'AP502F.A8EC.9724', 'AP502F.A8EC.9868', 'AP502F.A8EC.988C', 'AP502F.A8EC.9AAE', 'AP502F.A8EC.9B28', 'AP502F.A8EC.A052', 'AP502F.A8EC.A07E', 'AP502F.A8EC.A1B4', 'AP502F.A8EC.A214', 'AP7069.5A21.6CA6', 'AP7069.5A21.6CCA', 'AP7069.5A21.6CF4', 'AP7069.5A21.7106', 'AP7069.5a21.7264', 'AP7872.5D61.2310', 'AP7872.5D61.3120', 'AP7872.5D61.3128', 'AP7872.5D61.320A', 'AP7872.5D61.38E0', 'AP7872.5D61.3BEC', 'AP7872.5D61.40C6', 'AP7872.5D61.55C2', 'AP7872.5D61.5BAA', 'AP7872.5D62.18AE', 'AP7872.5D62.194A', 'AP7872.5D62.19A6', 'AP7872.5D62.25BC', 'AP7872.5D62.26A6', 'AP7872.5D62.C62C', 'AP7872.5D62.C662', 

In [40]:
# Eliminate unnecessary columns.
dropped = occ.drop(columns=list(filter(
    # add to drop-list if NOT in the buildings we want.
    lambda col: schema.col_to_building(col, safe=True) not in buildings,
    occ.columns.to_list()
)))

print(dropped.columns.to_list())

occ = dfm.fill_intervening_nas(
    occ, inplace=True, fill_val=0
)

['POM-ITB-113', 'POM-ITB-110', 'POM-ITB-115', 'POM-ALEXANDER-216', 'POM-ALEXANDER-217', 'POM-ALEXANDER-215', 'POM-ALEXANDER-219', 'POM-CLARKV-S23B', 'POM-CLARKV-460', 'POM-CLARKV-465', 'POM-ALEXANDER-137', 'POM-CARNEGIE-011', 'POM-CLARKI-12', 'POM-ALEXANDER-001', 'POM-MASON-207', 'POM-CLARKI-210', 'POM-CLARKI-131', 'POM-CLARKIII-121', 'POM-CLARKI-134', 'POM-SCC-121-1', 'POM-SCC-216', 'POM-SCC-18D', 'POM-SCC-18A', 'POM-CLARKV-2A', 'POM-SCC-218', 'POM-MASON-211', 'POM-CLARKI-221', 'POM-CARNEGIE-208', 'POM-CARNEGIE-203', 'POM-ITB-120', 'POM-SCC-155-2', 'POM-CLARKI-212', 'POM-CLARKV-418', 'POM-CLARKV-S3', 'POM-CLARKV-412', 'POM-CLARKIII-113', 'POM-SCC-134', 'POM-HAHN-218', 'POM-HAHN-214', 'POM-HAHN-216', 'POM-HAHN-210', 'POM-HAHN-212', 'POM-SCC-4', 'POM-SCC-148', 'POM-SCC-30-1', 'POM-SCC-30-2', 'POM-ALEXANDER-134', 'POM-MASON-005', 'POM-MASON-002', 'POM-SCC-240', 'POM-SCC-246', 'POM-MASON-022', 'POM-ITB-121-2', 'POM-ITB-121-1', 'POM-CARNEGIE-015', 'POM-ALEXANDER-122', 'POM-CARNEGIE-020', '

In [41]:
occ_bybuild = occ.groupby(
    by=schema.col_to_building, axis=1
)

print(occ_bybuild.groups)

Exception: No matching buildings for 'POM-MUDD-B02'